In [ ]:
import json
import pandas as pd



In [2]:

file_path="anime_2025_by_season.json"

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
            
except FileNotFoundError:
    print(f"File not found: {file_path}")
       
except json.JSONDecodeError:
    print("Invalid JSON format")
        

แปลงjson

In [33]:
rows = []

for season, anime_list in data['seasons'].items():
    for anime in anime_list:
        rows.append({
            "mal_id": anime["mal_id"],
            "url": anime["url"],
            "season": season,
            "year": data["year"],
            "trailer":anime['trailer'],
            "title": anime['title'],
            "title_english":anime["title_english"],
            "type" : anime['type'],
            "source":anime['source'],
            "rating":anime['rating'],
            "score" : anime['score'],
            "genres":anime['genres']
        })


In [35]:
display(rows )

[{'mal_id': 58567,
  'url': 'https://myanimelist.net/anime/58567/Ore_dake_Level_Up_na_Ken_Season_2__Arise_from_the_Shadow',
  'season': 'winter',
  'year': 2025,
  'trailer': {'youtube_id': None,
   'url': None,
   'embed_url': 'https://www.youtube-nocookie.com/embed/GDMXGzjJzS4?enablejsapi=1&wmode=opaque&autoplay=1',
   'images': {'image_url': None,
    'small_image_url': None,
    'medium_image_url': None,
    'large_image_url': None,
    'maximum_image_url': None}},
  'title': 'Ore dake Level Up na Ken Season 2: Arise from the Shadow',
  'title_english': 'Solo Leveling Season 2: Arise from the Shadow',
  'type': 'TV',
  'source': 'Web manga',
  'rating': 'R - 17+ (violence & profanity)',
  'score': 8.58,
  'genres': [{'mal_id': 1,
    'type': 'anime',
    'name': 'Action',
    'url': 'https://myanimelist.net/anime/genre/1/Action'},
   {'mal_id': 2,
    'type': 'anime',
    'name': 'Adventure',
    'url': 'https://myanimelist.net/anime/genre/2/Adventure'},
   {'mal_id': 10,
    'type